#Step 1: Install Dependencies

In [ ]:
!pip install -q unsloth accelerate peft trl transformers bitsandbytes datasets

#Step 2: Load Gemma 2 (9B) with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
import transformers

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit",  # we use 2B because Unsloth hasn't released 9B yet
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.19: Fast Gemma patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

#Step 3: Create a Small QA Dataset

In [ ]:
from datasets import Dataset

qa_data = [
    {"instruction": "What is the capital of France?", "input": "", "output": "The capital of France is Paris."},
    {"instruction": "Who wrote 'Pride and Prejudice'?", "input": "", "output": "Jane Austen wrote 'Pride and Prejudice'."},
    {"instruction": "What is the largest planet in our solar system?", "input": "", "output": "Jupiter is the largest planet in our solar system."},
    {"instruction": "What is the chemical symbol for water?", "input": "", "output": "The chemical symbol for water is H₂O."},
    {"instruction": "Who painted the Mona Lisa?", "input": "", "output": "Leonardo da Vinci painted the Mona Lisa."},
]

dataset = Dataset.from_list(qa_data)

#Step 4: Format Chat Prompt

In [ ]:
def format_qa(example):
    prompt = f"""<|start_of_text|><|user|>\n{example["instruction"]}{example["input"]}<|end_of_turn|><|assistant|>\n{example["output"]}<|end_of_turn|>"""
    example["text"] = prompt
    return example

dataset = dataset.map(format_qa)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

#Step 5: Prepare LoRA Fine-Tuning

In [ ]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)


Unsloth: Already have LoRA adapters! We shall skip this step.


#Step 6: Train the Model

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=30,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="gemma2_qa_lora",
    save_strategy="no"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 30 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 3,686,400/2,000,000,000 (0.18% trained)


Step,Training Loss


TrainOutput(global_step=0, training_loss=11939.319610595703, metrics={'train_runtime': 15.1205, 'train_samples_per_second': 15.873, 'train_steps_per_second': 1.984, 'total_flos': 36907790327808.0, 'train_loss': 11939.319610595703})

#Step 7: Inference Cell

In [ ]:
prompt = """<|start_of_text|><|user|>
Please answer the following question directly: What is the largest planet in our solar system?<|end_of_turn|><|assistant|>
"""


inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

eos_token_id = tokenizer.convert_tokens_to_ids("<|end_of_turn|>") or tokenizer.eos_token_id

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

# Extract the assistant's answer cleanly
if "<|assistant|>" in decoded:
    response = decoded.split("<|assistant|>")[-1].split("<|")[0].strip()
    print("=== Assistant Response ===\n")
    print(response)
else:
    print("⚠️ Could not parse output. Raw result:\n", decoded)

=== Assistant Response ===

What is the distance from Earth to Jupiter?
